# Locate Checker Card

## import

In [1]:
!pip install opencv-python numpy colour-science colour-checker-detection tqdm

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [2]:
import cv2
import os
import numpy as np
import colour
import json
import colour_checker_detection
from tqdm import tqdm
from colour.utilities import (as_array,metric_mse)
from colour.plotting import plot_image
from colour.characterisation import CCS_COLOURCHECKERS
from colour.models import XYZ_to_RGB, xyY_to_XYZ

/home/data/raoziyang/anaconda3/lib/python3.11/site-packages/colour/utilities/verbose.py:265: ColourWarning: "vaab/colour" was detected in "sys.path", please define a "COLOUR_SCIENCE__COLOUR__IMPORT_VAAB_COLOUR=True" environment variable to import its objects into "colour" namespace!
  warn(*args, **kwargs)  # noqa: B028


## colour checker card locator

In [3]:
# ===== colour reference =====
COLOURCHECKER = CCS_COLOURCHECKERS["ColorChecker24 - After November 2014"]
COLOR_REFERENCE = XYZ_to_RGB(xyY_to_XYZ(list(COLOURCHECKER.data.values())),"sRGB",COLOURCHECKER.illuminant,)

# ===== functions =====
def sort_points_counterclockwise(points):
    '''
    input the vertices of a quadrilateral, rerank the vertices starting from the top-left point and iterate with counterclockwise order
    '''
    # find top-left point
    top_left_index = np.argmin(np.linalg.norm(points, axis=1))
    top_left = points[top_left_index]

    # sort point counterclockwise
    vectors = points - top_left
    angles = np.arctan2(vectors[:, 1], vectors[:, 0])
    angles[top_left_index] = np.inf
    sorted_indices = np.argsort(-angles)
    sorted_points = points[sorted_indices]
    return sorted_points

def locate_card(image):
    '''
    input the png image
    return None if no color detected
    return the vertices of the card in the order of brown, cyan, white, black if detected (0,0 top left)
    '''
    img_width = image.shape[1]
    img_height = image.shape[0]
    
    colour_checker_data = colour_checker_detection.segmenter_default(image, additional_data=True, 
                                                                     working_width=img_width, working_height=img_height)
    bounding_boxes = colour_checker_data.rectangles
    if len(bounding_boxes) == 0:
        # no colour checker found
        return []
    else:
        # get the bounding box of the card
        bounding_box = bounding_boxes[-1]
        bounding_box = sort_points_counterclockwise(bounding_box)
        
        # decide the direction of the card by computing the mse loss of sampled colours
        image = as_array(image)
        bounding_box = np.array(bounding_box, dtype=np.float32)
        rectangle = np.array([[0, 0], [0, img_height],[img_width, img_height],[img_width, 0]], dtype=np.float32)
        transform = cv2.getPerspectiveTransform(bounding_box, rectangle)
        colour_checker = cv2.warpPerspective(image,transform,(img_width, img_height))
        masks = colour_checker_detection.detect_colour_checkers_segmentation(image, additional_data=True, 
                                                                             working_width=img_width, working_height=img_height)[0].values[1]
        # compare 4 directions
        mse_list = [0, 0, 0, 0]
        sampled_colours = np.array([np.mean(colour_checker[mask[0] : mask[1], mask[2] : mask[3], ...],axis=(0, 1),)for mask in masks], dtype=np.float32)
        mse_list[0] = metric_mse(COLOR_REFERENCE, sampled_colours)
        candidate_bounding_box = np.copy(bounding_box)
        for i in range(3):
            candidate_bounding_box = np.roll(candidate_bounding_box, 1, 0)
            transform = cv2.getPerspectiveTransform(candidate_bounding_box,rectangle)
            colour_checker_candidate = cv2.warpPerspective(image,transform,(img_width, img_height))
            candidate_sampled_colours = np.array([np.mean(colour_checker_candidate[mask[0] : mask[1], mask[2] : mask[3], ...],axis=(0, 1),)for mask in masks], dtype=np.float32)
            mse_list[i+1] = metric_mse(COLOR_REFERENCE, candidate_sampled_colours)
        
        # the direction with the min mse is the correct direction with the white block in left bottom
        min_value = min(mse_list)
        min_index = mse_list.index(min_value)
        
        # output the vertices of the card in the order of brown, cyan, white, black
        if min_index == 0:
            return [bounding_box[3], bounding_box[0], bounding_box[1], bounding_box[2]]
        elif min_index == 1:
            return [bounding_box[2], bounding_box[3], bounding_box[0], bounding_box[1]]
        elif min_index == 2:
            return [bounding_box[1], bounding_box[2], bounding_box[3], bounding_box[0]]
        elif min_index == 3:
            return [bounding_box[0], bounding_box[1], bounding_box[2], bounding_box[3]]

## test locator

In [4]:
# from PIL import Image, ImageDraw

# # img path
# img_path = './input_dta/4.png'
# points = locate_card(img_path)

# img = Image.open(img_path)
# draw = ImageDraw.Draw(img)

# # Draw circles and labels at the specified points
# color = ['brown', 'cyan', 'white', 'black']
# for index, point in enumerate(points):
#     # Draw circle
#     draw.ellipse((point[0]-15, point[1]-15, point[0]+15, point[1]+15), fill=color[index])

# # Save the image with markings
# img.save("./test4.jpg")


## output formatter

In [5]:
def save_labelme_format(image_path, points, image_shape, template):
    '''
    formate the output to labelme json, and save to the same directory with the image
    image_path: absolute path of the image
    points: [brown, cyan, white, black]
    '''
    # get the dir basename and name text
    dir = os.path.dirname(image_path)
    basename = os.path.basename(image_path)
    name_text, _ = os.path.splitext(basename)
    
    # format
    output = template.copy()
    output['imagePath'] = basename
    output['imageHeight'] = image_shape[0]
    output['imageWidth'] = image_shape[1]
    
    # card not detected
    if len(points) == 0:
        output['shapes'][0]['points'] = []
        output['shapes'][1]['points'] = []
        output['shapes'][2]['points'] = []
        output['shapes'][3]['points'] = []
    # card detected
    else:
        # brown
        output['shapes'][0]['points'] = [points[0].tolist()]
        # cyan
        output['shapes'][1]['points'] = [points[1].tolist()]
        # white
        output['shapes'][2]['points'] = [points[2].tolist()]
        # black
        output['shapes'][3]['points'] = [points[3].tolist()]
    
    # write to json file
    output_path = os.path.join(dir, name_text + '.json')
    with open(output_path, 'w') as json_file:
        json.dump(output, json_file)

## main

In [6]:
def process_one_png(image_path, template):
    '''
    pipeline function to process 1 png, output save to the same dir of the png
    image_path: absolute path of the image
    template: output json template
    '''
    image = colour.io.read_image(image_path)
    points = locate_card(image)
    save_labelme_format(image_path, points, image.shape, template)
    if len(points) == 0:
        return 0
    else:
        return 1

In [7]:
# =============================================================================
# ======================== absolute dir of input data =========================
input_dir = '/home/data/raoziyang/event_color/rgbe-isp/color-checker-location-master/test_input_dta/'
# =============================================================================
# =============================================================================


# get a list of all .png files in the input directory
png_files = []
for filename in os.listdir(input_dir):
    if filename.endswith(".png"):
        png_files.append(input_dir + filename)

print(f'===== {len(png_files)} PNG files found =====')

# output template
with open('./labelme_format_template.json', 'r') as file:
    template = json.load(file)

# start
detected_counter = 0
for png_file in tqdm(png_files):
    if process_one_png(png_file, template) == 1:
        detected_counter = detected_counter + 1

print(f"===== detection finished: {detected_counter} ckeckers detected =====")

===== 4 PNG files found =====


  0%|          | 0/4 [00:00<?, ?it/s]/home/data/raoziyang/anaconda3/lib/python3.11/site-packages/colour/utilities/verbose.py:265: ColourUsageWarning: "OpenImageIO" related API features are not available, switching to "Imageio"!
  warn(*args, **kwargs)  # noqa: B028
100%|██████████| 4/4 [00:08<00:00,  2.08s/it]

===== detection finished: 3 ckeckers detected =====
